Transactions (to be performed in Python):

1. **Task: Bank Transactions**
You are tasked with implementing a banking system that allows customers to transfer funds between accounts.
Design a SQL transaction to ensure that the amount is deducted from the sender's account and added to the receiver's account in an atomic manner.
If any part of the transaction fails, rollback the changes to maintain data integrity.

2. **Task: Order Processing**
You are developing an online shopping platform.
Create a SQL transaction that updates the inventory of products when a customer places an order.
Deduct the ordered quantity from the product stock and update the order status.
If any part of the transaction fails (e.g., insufficient stock), roll back the changes to ensure consistent data.

3. **Task: Employee Salary Adjustment**
Your HR department needs to apply a salary adjustment to all employees based on their performance.
Design a SQL transaction that updates the salary of each employee with the corresponding adjustment and logs the changes. If there is any issue while updating salaries, roll back the changes to maintain the previous salary data.

4. **Task: Flight Reservation**
You are building a flight reservation system. Create a SQL transaction that reserves a seat for a passenger on a flight.
Deduct the seat availability, update the reservation record, and log the booking details.
If any part of the transaction fails (e.g., seat already taken), roll back the changes and notify the user of the failure.

5. **Task: Product Returns**
You are developing an e-commerce platform. Implement a SQL transaction to process product returns initiated by customers.
Update the product's return status, refund the customer's payment, and log the return details.
If any part of the transaction fails (e.g., invalid return request), roll back the changes and handle the error gracefully.

6. **Task: Student Enrollment**
Design a SQL transaction to handle the enrollment of new students in a university.
Update the student enrollment records, assign courses, and update the tuition fee payment status. 
If any part of the transaction fails (e.g., course limit reached), roll back the changes to ensure no invalid enrollments are recorded.

Non SQL database:

In Python, design your own design of the database (using the files) and perform some practical examples with it. 

# Preparations for tasks: code automation

In [10]:
import mysql.connector

In [17]:
class db_updater:

    def __init__(self, db):
        self.db = db

    def sql_query(self, query, commit = True, fetch = False, insert_list = False, listt = None):
        try:
            conn = mysql.connector.connect(host='localhost',
                                        database= self.db,
                                        user='root',
                                        password='')

            conn.autocommit = False
            cursor = conn.cursor(buffered= True)

            # for inserting many rows in table automatically
            if insert_list == True:
                cursor.executemany(query, listt)
            else:
                cursor.execute(query)

            # for transaction
            if commit == True:
                conn.commit()

            # for gathering information, observations
            if fetch == True:
                result = cursor.fetchall()[0]
                columns = [col[0] for col in cursor.description]
                result_with_columns = [columns] + list(result)

                for row in result_with_columns:
                    print(row)

        except mysql.connector.Error as error:
            print("Failed to update record to database rollback: {}".format(error))
            conn.rollback()
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()
                print("connection is closed")

# 1st task

1. **Task: Bank Transactions**
You are tasked with implementing a banking system that allows customers to transfer funds between accounts.
Design a SQL transaction to ensure that the amount is deducted from the sender's account and added to the receiver's account in an atomic manner.
If any part of the transaction fails, rollback the changes to maintain data integrity.

In [8]:
btra = db_updater('bank_transactions_2023')

In [12]:
# create tables

query_users = btra.sql_query("""CREATE TABLE users
                             (account_id INT AUTO_INCREMENT PRIMARY KEY,
                             name VARCHAR(50),
                             surname VARCHAR(100),
                             birthdate DATE,
                             email VARCHAR(100),
                             balance INT);
                                """)

query_money_operations = btra.sql_query("""CREATE TABLE money_operations
                                        (timestamp TIMESTAMP PRIMARY KEY,
                                        sender INT,
                                        receiver INT,
                                        amount INT);
                                        """)

connection is closed
connection is closed


In [9]:
query1 = btra.sql_query("select * from users;", commit = False, fetch = True)

['account_id', 'name', 'surname', 'birthdate']
(1, 'test', 'tester', datetime.date(1998, 5, 31))
(2, 'hello', 'kitty', datetime.date(1995, 1, 3))
connection is closed


In [23]:
# example people
query_people = btra.sql_query("""INSERT INTO users (name, surname, birthdate, email, balance) VALUES
                              ("Billy", "Bill", "1996-12-25", "example@exampler.hyh", 12345),
                              ("Katniss", "Everdeen", "1994-04-15", "imhungry@games.lol", 100054),
                              ("Rich", "Richard", "1956-03-31", "richestofemall@million.cgf", 5);
                                        """)

connection is closed


In [23]:
def transfer_funds(sender_account, receiver_account, amount):
    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'bank_transactions_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()
        conn.start_transaction()

        cursor.execute("SELECT balance FROM users WHERE account_id = %s", (sender_account,))
        sender_balance = cursor.fetchone()[0]
        print(sender_balance)
        if sender_balance < amount:
            raise ValueError("Insufficient funds in the sender's account.")
        
        sender_balance -= amount
        cursor.execute("UPDATE users SET balance = %s WHERE account_id = %s", (sender_balance, sender_account))

        cursor.execute("SELECT balance FROM users WHERE account_id = %s", (receiver_account,))
        receiver_balance = cursor.fetchone()[0]
        
        receiver_balance += amount
        cursor.execute("UPDATE users SET balance = %s WHERE account_id = %s", (receiver_balance, receiver_account))
        
        cursor.execute("INSERT INTO money_operations VALUES (NOW(), {}, {}, {})".format(sender_account, receiver_account, amount))

        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()


In [24]:
# Example usage:
sender_account = '1'
receiver_account = '3'
amount_to_transfer = 100.0

transfer_funds(sender_account, receiver_account, amount_to_transfer)

700
Transfer successful.


# 2nd task

2. **Task: Order Processing**
You are developing an online shopping platform.
Create a SQL transaction that updates the inventory of products when a customer places an order.
Deduct the ordered quantity from the product stock and update the order status.
If any part of the transaction fails (e.g., insufficient stock), roll back the changes to ensure consistent data.

In [4]:
orpro = db_updater('order_processing_2023')

In [30]:
# create tables

query_products = orpro.sql_query("""CREATE TABLE products
                                (product_id INT AUTO_INCREMENT PRIMARY KEY,
                                product_name VARCHAR(50),
                                category VARCHAR(50),
                                price FLOAT,
                                amount INT);
                                """)

query_customers = orpro.sql_query("""CREATE TABLE customers
                                    (customer_id INT AUTO_INCREMENT PRIMARY KEY,
                                    name VARCHAR(50),
                                    surname VARCHAR(100),
                                    email VARCHAR(100));
                                    """)

query_money_orders = orpro.sql_query("""CREATE TABLE orders
                                        (order_id INT AUTO_INCREMENT PRIMARY KEY,
                                        customer_id INT,
                                        product_id INT,
                                        ordered_amount INT,
                                        status VARCHAR(30));
                                        """)

connection is closed
connection is closed
connection is closed


In [31]:
# example data

query_customers = orpro.sql_query("""INSERT INTO customers (name, surname, email) VALUES
                                ("Billy", "Bill", "example@exampler.hyh"),
                                ("Katniss", "Everdeen", "imhungry@games.lol"),
                                ("Rich", "Richard", "richestofemall@million.cgf");
                                """)

query_products = orpro.sql_query("""INSERT INTO products (product_name, category, price, amount) VALUES
                                ("Barbie doll", "Toys", 29.99, 12),
                                ("Keyboard 123", "Electronics", 99.99, 2),
                                ("Banana shaped pencil case", "Stationery", 6.99, 99),
                                ("Orange pen", "stationery", 0.69, 9001);
                                """)

connection is closed
connection is closed


In [3]:
def order_now():

    customer_account = input("Please write your customer id: ")
    product = input("Please write the ID of which product you would like to order: ")
    amount = input("How many products you would like you order? ")

    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'order_processing_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        conn.start_transaction()

        cursor.execute("SELECT amount FROM products WHERE product_id = %s", (product,))
        product_inventory = cursor.fetchone()[0]
        
        if product_inventory < int(amount):
            raise ValueError("Sorry, we do not have requested amount of items in stock.")
        
        product_inventory -= int(amount)
        cursor.execute("UPDATE products SET amount = %s WHERE product_id = %s", (product_inventory, product))
        
        cursor.execute("INSERT INTO orders (customer_id, product_id, ordered_amount, status) VALUES ({}, {}, {}, 'ordered')".format(customer_account, product, amount))

        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [34]:
#let's make some orders :)

order1 = order_now()

Transfer successful.


# 3rd task

3. **Task: Employee Salary Adjustment**
Your HR department needs to apply a salary adjustment to all employees based on their performance.
Design a SQL transaction that updates the salary of each employee with the corresponding adjustment and logs the changes. If there is any issue while updating salaries, roll back the changes to maintain the previous salary data.

In [36]:
sala = db_updater('salary_tracking_2023')

In [8]:
# create tables

query_employees = sala.sql_query("""CREATE TABLE employees
                                (employee_id INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(50),
                                surname VARCHAR(100),
                                salary FLOAT);
                                """)

query_tracking = sala.sql_query("""CREATE TABLE tracking
                                    (change_id INT AUTO_INCREMENT PRIMARY KEY,
                                    employee_id INT,
                                    timestamp TIMESTAMP,
                                    salary FLOAT);
                                    """)



Failed to update record to database rollback: 1050 (42S01): Table 'employees' already exists
connection is closed
connection is closed


In [7]:
# example data
# salary per month, in euro

query_example = sala.sql_query("""INSERT INTO employees (name, surname, salary) VALUES
                                ("Billy", "Bill", 2000),
                                ("Katniss", "Everdeen", 2500),
                                ("Rich", "Richard", 1005),
                                ("Example", "Examplier", 6000);
                                """)

connection is closed


In [37]:
def update_salary(employee_id, proposed_salary):

    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'salary_tracking_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        conn.start_transaction()

        cursor.execute("SELECT name FROM employees WHERE employee_id = %s", (employee_id,))
        row = cursor.fetchone()[0]
        if row is None:
            raise ValueError("No such employee found.")

        cursor.execute("SELECT salary FROM employees WHERE employee_id = %s", (employee_id,))
        salary_check = cursor.fetchone()[0]
        
        if salary_check < 600:
            raise ValueError("Sorry, employees' salaries must be higher than 600 due to regulations")
        
        cursor.execute("UPDATE employees SET salary = %s WHERE employee_id = %s", (proposed_salary, employee_id))
        
        cursor.execute("INSERT INTO tracking (employee_id, timestamp, salary) VALUES ({}, NOW(), {})".format(employee_id, proposed_salary))

        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [38]:
update1 = update_salary(2, 2358)

Transfer successful.


# 4th task

4. **Task: Flight Reservation**
You are building a flight reservation system. Create a SQL transaction that reserves a seat for a passenger on a flight.
Deduct the seat availability, update the reservation record, and log the booking details.
If any part of the transaction fails (e.g., seat already taken), roll back the changes and notify the user of the failure.

In [3]:
# creating seats

rows = [x for x in range(1, 39)]
columns = ["A", "B", "C", "D", "E", "F"]

seats = []
for row in rows:
    for column in columns:
        seat = (str(row) + column, "free")
        seats.append(seat)

In [4]:
flight = db_updater('flight_reservations_2023')

In [6]:
query_seats = flight.sql_query("""CREATE TABLE seats
                                (seat_id VARCHAR(5) PRIMARY KEY,
                                status VARCHAR(50));
                                """)


connection is closed


In [9]:
query_customers = flight.sql_query("""CREATE TABLE customers
                                (customer_id INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(50),
                                surname VARCHAR(100),
                                email VARCHAR(100));
                                """)


connection is closed
connection is closed


In [29]:
query_reservations = flight.sql_query("""CREATE TABLE reservations
                                    (reservation_id INT AUTO_INCREMENT PRIMARY KEY,
                                    seat_id VARCHAR(5),
                                    customer_id INT,
                                    timestamp TIMESTAMP);
                                    """)

connection is closed


In [8]:
#inserting list of seats created to the database
query_insert_seats = flight.sql_query("""INSERT INTO seats (seat_id, status) VALUES (%s, %s);""", insert_list= True, listt =seats)

connection is closed


In [10]:
query_insert_customers = flight.sql_query("""INSERT INTO customers (name, surname, email) VALUES
                                ("Billy", "Bill", "example@exampler.hyh"),
                                ("Katniss", "Everdeen", "imhungry@games.lol"),
                                ("Rich", "Richard", "richestofemall@million.cgf");
                                """)

connection is closed


In [40]:
import mysql.connector

def reserve_seats(customer_id, seat):
    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'flight_reservations_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        cursor.execute("SELECT status FROM seats WHERE seat_id = %s", (seat,))
        row = cursor.fetchone()
        if row is None:
            raise ValueError("Seat not found.")
        
        seat_status = row[0]
        if seat_status != "free":
            raise ValueError("Sorry, the seat is taken.")
        
        cursor.execute("UPDATE seats SET status = 'occupied' WHERE seat_id = %s", (seat,))
        cursor.execute("INSERT INTO reservations (seat_id, customer_id, timestamp) VALUES (%s, %s, NOW())", (seat, customer_id))

        conn.commit()

        print("Reservation successful.")
    except mysql.connector.Error as e:
        conn.rollback()
        print(f"Reservation failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [43]:
reserve_seats(2, "10B")

Reservation successful.


# 5th task

5. **Task: Product Returns**
You are developing an e-commerce platform. Implement a SQL transaction to process product returns initiated by customers.
Update the product's return status, refund the customer's payment, and log the return details.
If any part of the transaction fails (e.g., invalid return request), roll back the changes and handle the error gracefully.

In [ ]:
#going to use the database from the 2nd task since the topics overlap a lot, orders can be returned

#task outline: 1) add returns table 2)match request with the order 3)if matches commit, if not rollback

In [5]:
orpro = db_updater('order_processing_2023')

In [8]:
# making more orders so that there would be selection for returns
order1 = order_now()

Transfer successful.


In [7]:
query_returns = orpro.sql_query("""CREATE TABLE returns
                                    (return_id INT AUTO_INCREMENT PRIMARY KEY,
                                    order_id INT,
                                    customer_id INT,
                                    product_id INT,
                                    reason_id INT,
                                    item_amount INT,
                                    money_refunded FLOAT,
                                    timestamp TIMESTAMP);
                                    """)

query_return_reasons = orpro.sql_query("""CREATE TABLE return_reasons
                                    (reason_id INT AUTO_INCREMENT PRIMARY KEY,
                                    reason VARCHAR(300));
                                    """)

connection is closed
Failed to update record to database rollback: 1050 (42S01): Table 'return_reasons' already exists
connection is closed


In [40]:
reasons = ["Items broken", "Package lost", "Dissatisfaction with product within 14 days upon purchase", "Item does not match the description", "Different item arrived"]
reasoned = [(x,) for x in reasons]

insert_reasons = orpro.sql_query("""INSERT INTO return_reasons (reason) VALUES (%s);""", insert_list= True, listt =reasoned)

[('Items broken',), ('Package lost',), ('Dissatisfaction with product within 14 days upon purchase',), ('Item does not match the description',), ('Different item arrived',)]
connection is closed


In [24]:
def return_products(customer_id, product_id, amount, reason_id, order_id):

    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'order_processing_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        conn.start_transaction()

        cursor.execute("SELECT status FROM orders WHERE order_id = %s", (order_id,))
        row_ord = cursor.fetchone()
        if row_ord is None:
            raise ValueError("Invalid return, such order does not exist")
        
        cursor.execute("SELECT customer_id FROM orders WHERE order_id = %s", (order_id,))
        row_cust = cursor.fetchone()[0]
        if row_cust != customer_id:
            raise ValueError("Invalid return, return information does not match order data: customer")
        
        cursor.execute("SELECT product_id FROM orders WHERE order_id = %s", (order_id,))
        row_prod = cursor.fetchone()[0]
        if row_prod != product_id:
            raise ValueError("Invalid return, return information does not match order data: product")
        
        cursor.execute("SELECT ordered_amount FROM orders WHERE order_id = %s", (order_id,))
        row_amount = cursor.fetchone()[0]
        if row_amount != amount:
            raise ValueError("Invalid return, return information does not match order data: amount")
        
        cursor.execute("SELECT reason FROM return_reasons WHERE reason_id = %s", (reason_id,))
        row_reas = cursor.fetchone()[0]
        if row_reas is None:
            raise ValueError("Invalid reason for return: reason")
        
        cursor.execute("SELECT price FROM products WHERE product_id = %s", (product_id,))
        row_price = cursor.fetchone()[0]

        total_price = row_price * amount
        
        cursor.execute("""INSERT INTO returns (order_id, customer_id, product_id, reason_id, item_amount, money_refunded, timestamp) VALUES 
                       ({}, {}, {}, {}, {}, {}, NOW())""".format(order_id, customer_id, product_id, reason_id, amount, total_price))
        
        cursor.execute("UPDATE orders SET status = 'refunded' WHERE order_id = %s", (order_id,))

        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [23]:
return_products(1, 1, 1, 1, 1)

('ordered',)
1
1
Items broken
29.99
29.99
Transfer successful.


# 6th task

6. **Task: Student Enrollment**
Design a SQL transaction to handle the enrollment of new students in a university.
Update the student enrollment records, assign courses, and update the tuition fee payment status. 
If any part of the transaction fails (e.g., course limit reached), roll back the changes to ensure no invalid enrollments are recorded.

In [12]:
stud = db_updater('student_enrollment_2023')

In [5]:
query_students = stud.sql_query("""CREATE TABLE students
                                (student_id INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(50),
                                surname VARCHAR(100),
                                credits_taken INT);
                                    """)

query_courses = stud.sql_query("""CREATE TABLE courses
                                (course_id INT AUTO_INCREMENT PRIMARY KEY,
                                course_name VARCHAR(100),
                                student_no_limit INT,
                                credits INT);
                                """)

query_choices = stud.sql_query("""CREATE TABLE course_choices
                                (choice_id INT AUTO_INCREMENT PRIMARY KEY,
                                student_id INT,
                                course_id INT);
                                """)       

query_payment = stud.sql_query("""CREATE TABLE fee_payment
                                (payment_id INT AUTO_INCREMENT PRIMARY KEY,
                                student_id INT,
                                paid VARCHAR(5));
                                """)                          

Failed to update record to database rollback: 1050 (42S01): Table 'students' already exists
connection is closed
Failed to update record to database rollback: 1050 (42S01): Table 'courses' already exists
connection is closed
connection is closed
connection is closed


In [13]:
def add_student(name, surname, fee_type = 'no sponsor'):

    fee_types = ['no sponsor', 'full scholarship', 'partial scholarship', 'loan']
    if fee_type not in fee_types:
        raise ValueError("Invalid tuition fee payment type. Expected one of: %s", (fee_types))
    
    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'student_enrollment_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        conn.start_transaction()

        cursor.execute("INSERT INTO students (name, surname, credits_taken) VALUES (%s, %s, 0);", (name, surname,))
    
        cursor.execute("SELECT student_id FROM students WHERE name= %s AND surname = %s", (name, surname,))
        row_stud = cursor.fetchone()[0]
    
        if fee_type == "full scholarship":
            cursor.execute("INSERT INTO fee_payment (student_id, paid) VALUES (%s, 'yes');", (row_stud,))
        else:
            cursor.execute("INSERT INTO fee_payment (student_id, paid) VALUES (%s, 'no');", (row_stud,))
        
        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [10]:
student_names = ["Inga", "Julia", "Chris", "Ken", "Aliya", "Thomas", "Justin", "Cassandra"]
student_surnames = ["Gonzalez", "Pipiro", "Goblin", "Katty", "Examplier", "Capitol", "Zymus", "Antrapa"]
fees = ['no sponsor', 'full scholarship', 'partial scholarship', 'loan', 'no sponsor', 'full scholarship', 'partial scholarship', 'loan']

for i in range(len(student_names)):
    add_student(student_names[i], student_surnames[i], fees[i])

Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.


In [9]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
names = [
    "Mia", "Ethan", "Olivia", "Liam", "Ava", 
    "Noah", "Sophia", "Jackson", "Isabella", "Aiden", 
    "Harper", "Lucas", "Amelia", "Oliver", "Emma", 
    "Elijah", "Charlotte", "Carter", "Mia", "Jack", 
    "Sophia", "Benjamin", "Evelyn", "Daniel", "Scarlett", 
    "Matthew", "Grace", "Alexander", "Chloe", "Samuel", 
    "Lily", "Michael", "Abigail", "William", "Emily", 
    "James", "Ella", "Logan", "Elizabeth", "David", 
    "Victoria", "Joseph", "Madison", "Henry", "Avery", 
    "Gabriel", "Zoey", "Christopher", "Penelope", "Andrew"
]

surnames = [
    "Johnson", "Martinez", "Smith", "Anderson", "Williams", 
    "Thompson", "Davis", "Taylor", "Hernandez", "Brown", 
    "Miller", "Wilson", "Garcia", "Lee", "Rodriguez", 
    "Martinez", "Jackson", "White", "Adams", "Lewis", 
    "Hall", "Scott", "Young", "Turner", "Moore", 
    "Mitchell", "Nelson", "King", "Hill", "Green", 
    "Carter", "Brooks", "Wright", "Parker", "Ramirez", 
    "Kelly", "Foster", "Cooper", "Hayes", "Bennett", 
    "Flores", "Reed", "Rivera", "Price", "Coleman", 
    "Rogers", "Ward", "Ross", "Evans", "Powell"
]

fee = []
for i in range(50):
    fee.append('no sponsor')

In [14]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
for i in range(len(names)):
    add_student(names[i], surnames[i], fee[i])

Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer succ

In [8]:
query_insert_courses = stud.sql_query("""INSERT INTO courses (course_name, student_no_limit, credits) VALUES
                                    ("Linear Algebra", 15, 5),
                                    ("Molecular Biotechnology", 20, 5),
                                    ("Spectroscopy", 16, 10),
                                    ("Quantum Physics", 25, 5),
                                    ("Enzymology", 24, 10),
                                    ("Stereochemistry", 15, 5),
                                    ("Microbiology", 5, 10),
                                    ("Cell Natural and Artificial Lysis Pathways", 30, 10);
                                    """)

connection is closed


In [19]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
# Adding more courses
query_insert_courses = stud.sql_query("""INSERT INTO courses (course_name, student_no_limit, credits) VALUES
                                    ("Economics", 20, 5),
                                    ("Physics", 25, 10),
                                    ("Panel readings", 30, 5),
                                    ("Electromechanics", 17, 10),
                                    ("Psychology", 23, 5),
                                    ("Biochemistry", 20, 15),
                                    ("Analytical Chemistry", 25, 5),
                                    ("Discrete Mathematics", 30, 10);
                                    """)

connection is closed


In [15]:
def choose_course(student_id, course_id):
    try:
        conn = mysql.connector.connect(host='localhost',
                                        database= 'student_enrollment_2023',
                                        user='root',
                                        password='')
        
        cursor = conn.cursor()

        conn.start_transaction()

        cursor.execute("SELECT name FROM students WHERE student_id = %s", (student_id,))
        row_ch1 = cursor.fetchone()
        if row_ch1 is None:
            raise ValueError("Invalid choice, such student does not exist")
        
        cursor.execute("SELECT course_name FROM courses WHERE course_id = %s", (course_id,))
        row_ch2 = cursor.fetchone()
        if row_ch2 is None:
            raise ValueError("Invalid choice, such course does not exist")

        cursor.execute("SELECT credits_taken FROM students WHERE student_id = %s", (student_id,))
        row_credits = cursor.fetchone()[0]

        cursor.execute("SELECT credits FROM courses WHERE course_id = %s", (course_id,))
        row_course = cursor.fetchone()[0]

        if (row_credits + row_course) > 30:
            raise ValueError("Invalid choice, chosen moren than 30 credits for a semester")
        
        cursor.execute("""INSERT INTO course_choices (student_id, course_id) VALUES 
                       ({}, {})""".format(student_id, course_id))
        
        new_credits = row_credits + row_course
        
        cursor.execute("UPDATE students SET credits_taken = %s WHERE student_id = %s", (new_credits, student_id,))

        conn.commit()

        print("Transfer successful.")
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
    finally:
        cursor.close()
        conn.close()

In [ ]:
choose_course(2, 3)

In [18]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
st_id = stud.sql_query("SELECT student_id FROM students;", commit = False, fetch = True)
st_id
#we see that there are 59 students in total

['student_id']
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)
(24,)
(25,)
(26,)
(27,)
(28,)
(29,)
(30,)
(31,)
(32,)
(33,)
(34,)
(35,)
(36,)
(37,)
(38,)
(39,)
(40,)
(41,)
(42,)
(43,)
(44,)
(45,)
(46,)
(47,)
(48,)
(49,)
(50,)
(51,)
(52,)
(53,)
(54,)
(55,)
(56,)
(57,)
(58,)
(59,)
connection is closed


In [20]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
course_id = stud.sql_query("SELECT course_id FROM courses;", commit = False, fetch = True)
course_id
#we see that there are 16 students in total

['course_id']
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
connection is closed


In [35]:
# This part is for August 3 task! All added/edited parts for this tasks will be noted in the comments like this
# Will run this for a few times till all students have chosen 30 credits worth of studies
import random
for a in range(1, 60):
    choice = random.randint(1, 16)
    choose_course(a, choice)

Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester
Transfer failed: Invalid choice, chosen moren than 30 credits for a semester

In [16]:
#just paying fees when the time comes

def pay_the_fees(student_id):
    fee_paid = stud.sql_query("UPDATE fee_payment SET paid = 'yes' WHERE student_id = {}".format(student_id))
    return fee_paid

In [17]:
pay_the_fees(2)

connection is closed


# 7th task

Non SQL database:

In Python, design your own design of the database (using the files) and perform some practical examples with it. 

In [18]:
import json

In [19]:
class Winnings:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = self.load_data()

    def load_data(self):
        try:
            with open(self.file_path, 'r') as file:
                data = json.load(file)
                return data
        except FileNotFoundError:
            return []

    def commit(self):
        with open(self.file_path, 'w') as file:
            json.dump(self.data, file)

    def fetch(self, name, surname):
        for entry in self.data:
            if entry['name'] == name and entry['surname'] == surname:
                return entry
        return None

    def update(self, name, surname, medals, time):
        for entry in self.data:
            if entry['name'] == name and entry['surname'] == surname:
                entry['medals'] = medals
                entry['time'] = time
                self.commit()
                return True
        return False

    def insert(self, name, surname, age, medals, time):
        entry = {
            'name': name,
            'surname': surname,
            'age': age,
            'medals': medals,
            'time': time
        }
        self.data.append(entry)
        self.commit()
        return True

    def delete(self, name, surname):
        for entry in self.data:
            if entry['name'] == name and entry['surname'] == surname:
                self.data.remove(entry)
                self.commit()
                return True
        return False

    def get_last_item(self):
        if self.data:
            return self.data[-1]
        return None



In [27]:

db = Winnings("winnings_database.json")


In [28]:

db.insert("Janina", "Queue", 30, ["Gold", "Silver"], "2023")
db.insert("Janet", "Jaxon", 28, ["Bronze"], "2022")
db.insert("Alexandra", "Johns", 35, ["Gold", "Gold", "Silver"], "2023")


True

In [30]:
print(db.fetch("Janina", "Queue"))

{'name': 'Janina', 'surname': 'Queue', 'age': 30, 'medals': ['Gold', 'Silver'], 'time': '2023'}


In [31]:
print(db.get_last_item())

{'name': 'Alexandra', 'surname': 'Johns', 'age': 35, 'medals': ['Gold', 'Gold', 'Silver'], 'time': '2023'}


In [32]:
db.update("Janet", "Jaxon", ["Bronze", "Gold", "Silver"], ["2022", "2023", "2023"])
print(db.fetch("Janet", "Jaxon"))


{'name': 'Janet', 'surname': 'Jaxon', 'age': 28, 'medals': ['Bronze', 'Gold', 'Silver'], 'time': ['2022', '2023', '2023']}


In [33]:
db.delete("Janina", "Queue")
print(db.fetch("Janina", "Queue"))

None


In [34]:
db.commit()

In [35]:
db.load_data()

[{'name': 'Janet',
  'surname': 'Jaxon',
  'age': 28,
  'medals': ['Bronze', 'Gold', 'Silver'],
  'time': ['2022', '2023', '2023']},
 {'name': 'Alexandra',
  'surname': 'Johns',
  'age': 35,
  'medals': ['Gold', 'Gold', 'Silver'],
  'time': '2023'}]